In [ ]:
# References:
# Kai Tao, Tianze Liu, Jieyuan Ning, Fenglin Niu, "Estimating sedimentary and crustal structure
# using wavefield continuation: theory, techniques and applications", Geophysical Journal International,
# Volume 197, Issue 1, April, 2014, Pages 443-457, https://doi.org/10.1093/gji/ggt515

--------------------------------------------------------

# Functions to codify the algorithm

In [ ]:
def mode_matrices(Vp, Vs, rho, p):
    """Compute M, M_inv and Q for a single layer for a scalar or array of ray parameters p.
    
    :param Vp: P-wave body wave velocity (scalar, labeled α in Tao's paper)
    :type Vp: 
    :param Vs: S-wave body wave velocity (scalar, labeled β in Tao's paper)
    :type Vs: 
    :param rho: Bulk material density, ρ (scalar)
    :type rho: 
    :param p: Scalar or array of ray parameters (one per event)
    :type p: 
    """
    qa = np.sqrt((1/Vp**2 - p*p).astype(np.complex))
    assert not np.any(np.isnan(qa)), qa
    qb = np.sqrt((1/Vs**2 - p*p).astype(np.complex))
    assert not np.any(np.isnan(qb)), qb
    eta = 1/Vs**2 - 2*p*p
    mu = rho*Vs*Vs
    trp = 2*mu*p*qa
    trs = 2*mu*p*qb
    mu_eta = mu*eta
    # First compute without velocity factors for reduced operation count.
    M = np.array([
        [p, p, qb, qb],
        [qa, -qa, -p, p],
        [-trp, trp, -mu_eta, mu_eta],
        [-mu_eta, -mu_eta, trs, trs]
    ])
    # Then times by velocity factors
    Vfactors = np.diag([Vp, Vp, Vs, Vs])
    M = np.matmul(np.moveaxis(M, -1, 0), Vfactors)
    
    Q = np.dstack([np.expand_dims(np.array([-_1, _1, -_2, _2]), 1) for (_1, _2) in zip(qa, qb)])
    Q = np.moveaxis(Q, -1, 0)

    # First compute without velocity factors for reduced operation count.
    mu_p = mu*p
    Minv = (1.0/rho)*np.array([
        [mu_p, mu_eta/2/qa, -p/2/qa, -0.5*np.ones(p.shape)],
        [mu_p, -mu_eta/2/qa, p/2/qa, -0.5*np.ones(p.shape)],
        [mu_eta/2/qb, -mu_p, -0.5*np.ones(p.shape), p/2/qb],
        [mu_eta/2/qb, mu_p, 0.5*np.ones(p.shape), p/2/qb]
    ])
    # Then times by velocity factors
    Vfactors_inv = np.diag([1/Vp, 1/Vp, 1/Vs, 1/Vs])
    Minv = np.matmul(Vfactors_inv, np.moveaxis(Minv, -1, 0))
    
#     # DEBUG CHECK - verify M*Minv is close to identity
#     for i in range(M.shape[0]):
#         _M = M[i,:,:]
#         _Minv = Minv[i,:,:]
#         assert _M.shape[0] == _M.shape[1]
#         assert np.allclose(np.matmul(_M, _Minv).flatten(), np.eye(_M.shape[0]).flatten()), i
    
    return (M, Minv, Q)
# end func

In [ ]:
class LayerProps():
    # Helper class to contain layer bulk material properties
    def __init__(self, vp, vs, rho, thickness):
        self.Vp = vp
        self.Vs = vs
        self.rho = rho
        self.H = thickness # H value here is thickness of the individual layer, NOT depth relative to surface
    # end func
    
    def __repr__(self):
        return '(' + ', '.join(['Vp=' + str(self.Vp),
                                'Vs=' + str(self.Vs),
                                'ρ=' + str(self.rho),
                                'H=' + str(self.H)]) + ')'
    # end func

# end class

In [ ]:
def propagate_layers(fv0, w, layer_props, p):
    """
    layer_props is a list of LayerProps
    """
    fz = np.hstack((fv0, np.zeros_like(fv0)))
    for layer in layer_props:
        M, Minv, Q = mode_matrices(layer.Vp, layer.Vs, layer.rho, p)
        fz = np.matmul(Minv, fz)
#         phase_args = np.outer(Q - Q[1], w)
#         phase_args = np.matmul(Q, np.expand_dims(w, 0))
        # Expanding dims on w here means that at each level of the stack, phase_args is np.outer(Q, w)
        phase_args = np.matmul(Q, np.expand_dims(np.expand_dims(w, 0), 0))
        assert np.allclose(np.outer(Q[0,:,:], w).flatten(), phase_args[0,:,:].flatten()), (Q, w)
        phase_factors = np.exp(1j*layer.H*phase_args)
        fz = phase_factors*fz  # point-wise multiplication
        fz = np.matmul(M, fz)
    # end for
    return fz
# end func

In [ ]:
def compute_su_energy(v0, f_s, p, mantle_props, layer_props,
                      time_window=(-20, 50), flux_window=(-10, 20)):
    """Compute upgoing S-wave energy for a given set of seismic time series v0.
    
    :param v0: Numpy array of shape (N_events, 2, N_samples) containing the R- and
        Z-component traces for all events at sample rate f_s and covering duration
        of time_window.
    :param mantle_props: LayerProps representing mantle properties.
    :param layer_props: List of LayerProps.
    """
    dt = 1.0/f_s
    npts = v0.shape[2]
    nevts = v0.shape[0]
    t = np.linspace(*time_window, npts)

    # Reshape to facilitate max_vz normalization using numpy broadcast rules.
    v0 = np.moveaxis(v0, 0, -1)

    # Normalize each event signal by the maximum z-component amplitude.
    # We perform this succinctly using numpy multidimensional broadcasting rules.
    max_vz = np.abs(v0[1,:,:]).max(axis=0)
    v0 = v0/max_vz

    # Reshape back to original shape.
    v0 = np.moveaxis(v0, -1, 0)

    # Transform v0 to the spectral domain using real FFT
    fv0 = np.fft.rfft(v0, axis=-1)

    # Compute discrete frequencies
    w = 2*np.pi*np.fft.rfftfreq(v0.shape[-1], dt)

    # Extend w to full spectral domain.
    w_full = np.hstack((w, -np.flipud(w[1:])))

    # To extend fv0, we need to flip left-right and take complex conjugate.
    fv0_full = np.dstack((fv0, np.fliplr(np.conj(fv0[:, :, 1:]))))

    # Compute mode matrices for mantle
    M_m, Minv_m, _ = mode_matrices(mantle_props.Vp, mantle_props.Vs, mantle_props.rho, p)

    # Propagate from surface
    fvm = propagate_layers(fv0_full, w_full, layer_props, p)
    fvm = np.matmul(Minv_m, fvm) 

    num_pos_freq_terms = (fvm.shape[2] + 1)//2
    # Velocities at top of mantle
    vm = np.fft.irfft(fvm[:, :, :num_pos_freq_terms], v0.shape[2], axis=2)

    # Compute coefficients of energy integral for upgoing S-wave
    qb_m = np.sqrt(1/mantle_props.Vs**2 - p*p)
    Nsu = dt*mantle_props.rho*(mantle_props.Vs**2)*qb_m

    # Compute mask for the energy integral time window
    integral_mask = (t >= flux_window[0]) & (t <= flux_window[1])
    vm_windowed = vm[:, :, integral_mask]

    # Take the su component.
    su_windowed = vm_windowed[:, 3, :]

    # Integrate in time
    Esu_per_event = Nsu*np.sum(np.abs(su_windowed)**2, axis=1)

    # Compute mean over events
    Esu = np.mean(Esu_per_event)

    return Esu, Esu_per_event, vm
# end func

In [ ]:
def _streamdict_to_array(data, f_s, time_window, cut_window):
    """
    Convert dict of streams (indexed by event id) to numpy array in format required
    by function compute_su_energy(), including resampling to f_s and applying a cut
    window and sinc resampling.

    This conversion may be expensive and compute_su_energy() may need to be called
    many times, so preconverting the format to numpy array once only is important
    to overall performance.
    
    Any quality filtering needs to be performed prior to calling this function.
    
    Note: This function modifies in-place the traces in the values of data.
    """
    from seismic.receiver_fn.rf_util import sinc_resampling

    # Resample to f_s if any trace is not already as f_s
    for evid, stream in data.items():
        if np.any(np.array([tr.stats.sampling_rate != f_s for tr in stream])):
            # Resampling lowpass only, as per Tao (anti-aliasing)
            stream.filter('lowpass', freq=f_s/2.0, corners=2, zerophase=True).interpolate(
                          f_s, method='lanczos', a=10)
        # end if
    # end for

    # Trim to time window
    for evid, stream in data.items():
        stream.trim(stream[0].stats.onset + time_window[0],
                    stream[0].stats.onset + time_window[1])
    # end for

    # Cut central data segment and resample back to original length using sinc interpolation.
    for stream in data.values():
        for tr in stream:
            times = tr.times() - (tr.stats.onset - tr.stats.starttime)
            tr_cut = tr.copy().trim(tr.stats.onset + cut_window[0], tr.stats.onset + cut_window[1])
            tr_cut.detrend('linear')
            tr_cut.taper(0.10)
            cut_times = tr_cut.times() - (tr_cut.stats.onset - tr_cut.stats.starttime)
            cut_data = tr_cut.data
            resampled_data = sinc_resampling(cut_times, cut_data, times)
            # Replace trace data with cut resampled data
            tr.data = resampled_data
        # end for
    # end for

    # Pull data arrays out into matrix format
    v0 = np.array([[  st.select(component='R')[0].data.tolist(),
                    (-st.select(component='Z')[0].data).tolist()] for st in data.values()])

    return v0
# end func

In [ ]:
def job_caller(i, j, v0, f_s, p, mantle, earth_model, flux_window):
    energy, _, _ = compute_su_energy(v0, f_s, p, mantle, earth_model, flux_window=flux_window)
    return (i, j, energy)

In [ ]:
def find_energy_minimum_location(energy, h_grid, vs_grid):
    min_loc = np.unravel_index(np.argmin(energy), energy.shape)
    H_min = h_grid[0, min_loc[1]]
    Vs_min = vs_grid[min_loc[0], 0]
    return (H_min, Vs_min)
# end func

--------------------------------

# Import required libraries and run case studies

In [ ]:
from collections import defaultdict, OrderedDict
import logging

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import numpy.fft as fft
from scipy import stats

import h5py
import obspy
import obspyh5

from seismic.receiver_fn.stream_quality_filter import curate_stream3c
from seismic.receiver_fn.rf_util import compute_vertical_snr
from seismic.receiver_fn.rf_util import KM_PER_DEG
from seismic.receiver_fn.rf_util import sinc_resampling
from seismic.receiver_fn.rf_synthetic import synthesize_ideal_seismogram

In [ ]:
from tqdm.auto import tqdm
from joblib import Parallel, delayed

-----------------------------

## Run using on-demand synthetic data to validate implementation

In [ ]:
network = 'AU'
target_station = 'QIS'

In [ ]:
f_s = 10
data_synth = synthesize_ideal_seismogram(network, target_station, 'velocity', 65, 140, f_s=f_s,
                                         sourcedepthmetres=0)
# data_synth

In [ ]:
data_synth.plot(type='relative', reftime=data_synth[0].stats.onset, outfile='synth_event.png', dpi=300)

In [ ]:
# Time window to trim input traces to
TIME_WINDOW = (-20, 50)
# Snippet around onset to use for processing
CUT_WINDOW = (-5, 30)
# Time window used for integration of energy flux
FLUX_WINDOW = (-10, 20)

In [ ]:
t_onset = data_synth[0].stats.onset - data_synth[0].stats.starttime

In [ ]:
data_all = {'synth_event_0': data_synth}

In [ ]:
v0 = _streamdict_to_array(data_all, f_s, TIME_WINDOW, CUT_WINDOW)

In [ ]:
# Get ray params for the events
p = np.array([st[0].stats.slowness/KM_PER_DEG for st in data_all.values()])

In [ ]:
# Define bulk properties of mantle (lowermost half-space)
mantle_props = LayerProps(8.0, 4.5, 3.3, np.Infinity)

In [ ]:
# Define single layer earth model (crust over mantle only, no sediment)
# Vs here is postulated.
# H here is postulated.
earth_props = np.array([LayerProps(6.1, 3.7, 2.7, 35.0)])

In [ ]:
energy, energy_per_event, mantle_wave_components = compute_su_energy(v0, f_s, p, mantle_props, earth_props,
                                                                    flux_window=FLUX_WINDOW)

In [ ]:
print(energy)

In [ ]:
# Plot the wavefield decomposition at the top of the mantle
plt.figure(figsize=(16,12))
t = np.arange(v0.shape[2])/f_s - t_onset
plt.plot(t, mantle_wave_components[0,0,:], label='$P_{down}$', alpha=0.8, linewidth=2)
plt.plot(t, mantle_wave_components[0,1,:], label='$P_{up}$', alpha=0.8, linewidth=2)
plt.plot(t, mantle_wave_components[0,2,:], label='$S_{down}$', alpha=0.8, linewidth=2)
plt.plot(t, mantle_wave_components[0,3,:], label='$S_{up}$', alpha=0.8, linewidth=2)
plt.plot(t, np.sum(mantle_wave_components[0,:,:], axis=0), color='#40404080', label='Total', linewidth=4)
plt.xlabel('Time (s)', fontsize=16)
plt.ylabel('Amplitude (normalized)', fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(linestyle=':', color="#80808080")
plt.title("Waveform at top of mantle", fontsize=20, y=1.01)
plt.legend(fontsize=14)
plt.savefig('synth_waveform_top_mantle.png', dpi=300)
plt.show()

In [ ]:
# Define grid search space
H, Vs = np.meshgrid(np.linspace(25, 50, 121), np.linspace(3.2, 4.0, 81))
Esu = np.zeros(H.shape)

In [ ]:
# Run grid search and collect results
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Outer loop'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [LayerProps(6.1, _Vs, 2.7, _H)], FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

In [ ]:
# Plot energy flux across search space
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=50, cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=10, colors='k', linewidths=1, antialiased=True)
plt.xlabel('$V_s$ (km/s)', fontsize=14)
plt.ylabel('$H$ Moho depth (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('{}.{} synthetic'.format(network, target_station), fontsize=20, y=1.05)
plt.savefig('AU.QIS_synth_energy_flux.png', dpi=300)
plt.show()

In [ ]:
find_energy_minimum_location(Esu, H, Vs)

------------------------

## Run on data converted from Tao's SAC files

Replicate work of Tao on NE68

In [ ]:
network = 'BD'
target_station = 'NE68'

In [ ]:
# Resampling rate
f_s = 10.0  # Matches dt==0.1 used by Tao
# Time window of original data to use for processing. All traces must have at least this extent
# about the onset time.
TIME_WINDOW = (-20, 50)
# Narrower time window used for integration of energy flux
FLUX_WINDOW = (-10, 20)
# Cut window for selecting central wavelet
CUT_WINDOW = (-5, 30)

In [ ]:
# src_file = (r"/g/data/ha3/am7399/dev/RFsediment/YP.NE68/H-beta_SCM_Esu_DCmatlab_station/sac"
#             r"/event_test3.use.hdf5")
src_file = (r"/g/data/ha3/am7399/dev/RFsediment/YP.NE68/H-beta_SCM_Esu_DCmatlab_station/sac"
            r"/event.use.h5")

In [ ]:
traces = obspy.read(src_file, 'H5')

In [ ]:
# Group triplets of traces for same event id
data_all = defaultdict(obspy.Stream)
for tr in traces:
    data_all[tr.stats.event_id].append(tr.copy())
data_all = OrderedDict(sorted(data_all.items(), key=lambda k: k[0]))

In [ ]:
len(data_all)

In [ ]:
v0 = _streamdict_to_array(data_all, f_s, TIME_WINDOW, CUT_WINDOW)

In [ ]:
# Get ray params for the events
p = np.array([st[0].stats.slowness/KM_PER_DEG for st in data_all.values()])
print(p)

In [ ]:
# Define bulk properties of mantle (lowermost half-space)
mantle_props = LayerProps(8.0, 4.5, 3.3, np.Infinity)
mantle_props

In [ ]:
earth_props = np.array([LayerProps(2.1, 0.5, 1.97, 0.3), LayerProps(6.4, 3.7, 2.7, 35.0)])
earth_props

In [ ]:
energy, energy_per_event, mantle_wave_components = compute_su_energy(v0, f_s, p, mantle_props, earth_props,
                                                                     flux_window=FLUX_WINDOW)

In [ ]:
print(energy)

### Perform grid search on sediment properties

In [ ]:
import time

In [ ]:
total_time = 0

In [ ]:
crust_props = LayerProps(6.1, 3.7, 2.7, 35)

In [ ]:
H, Vs = np.meshgrid(np.linspace(0, 1, 101), np.linspace(0.1, 1.0, 91))
Esu = np.zeros(H.shape)

In [ ]:
t0 = time.perf_counter()
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Sediment loop 1'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [LayerProps(2.1, _Vs, 1.97, _H), crust_props],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

total_time += time.perf_counter() - t0

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=np.linspace(0.3, 1.80, 50), cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=np.linspace(0.3, 0.9, 10), colors='k', linewidths=1, antialiased=True)
plt.xlabel('Sediment $V_s$ (km/s)', fontsize=14)
plt.ylabel('Sediment $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('NE68 Sediment properties (1st iteration)', fontsize=20, y=1.05)
plt.savefig('NE68_sediment_props_iteration1.png', dpi=300)
plt.show()

In [ ]:
H_sediment, Vs_sediment = find_energy_minimum_location(Esu, H, Vs)
print(H_sediment, Vs_sediment)

### Perform grid search on crust properties

In [ ]:
sediment_props = LayerProps(2.1, Vs_sediment, 1.97, H_sediment)
sediment_props

In [ ]:
H, Vs = np.meshgrid(np.linspace(25, 45, 201), np.linspace(3.0, 5.0, 201))
Esu = np.zeros(H.shape)

In [ ]:
t0 = time.perf_counter()
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Crust loop'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [sediment_props, LayerProps(6.1, _Vs, 2.7, _H)],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

total_time += time.perf_counter() - t0

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=50, cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=10, colors='k', linewidths=1, antialiased=True)
plt.xlabel('Crust $V_s$ (km/s)', fontsize=14)
plt.ylabel('Crust $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('NE68 Crust properties (1st iteration)', fontsize=20, y=1.05)
plt.savefig('NE68_crust_props_iteration1.png', dpi=300)
plt.show()

In [ ]:
H_crust, Vs_crust = find_energy_minimum_location(Esu, H, Vs)
print(H_crust, Vs_crust)

### Repeat grid search on sediment properties (2nd iteration)

In [ ]:
crust_props = LayerProps(6.1, Vs_crust, 2.7, H_crust)

In [ ]:
H, Vs = np.meshgrid(np.linspace(0, 1, 101), np.linspace(0.1, 1.0, 91))
Esu = np.zeros(H.shape)

In [ ]:
t0 = time.perf_counter()
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Sediment loop'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [LayerProps(2.1, _Vs, 1.97, _H), crust_props],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

total_time += time.perf_counter() - t0

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=np.linspace(0.28, 1.8, 50), cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=np.linspace(0.28, 0.9, 10), colors='k', linewidths=1, antialiased=True)
plt.xlabel('Sediment $V_s$ (km/s)', fontsize=14)
plt.ylabel('Sediment $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('NE68 Sediment properties (2nd iteration)', fontsize=20, y=1.05)
plt.savefig('NE68_sediment_props_iteration2.png', dpi=300)
plt.show()

In [ ]:
H_sediment, Vs_sediment = find_energy_minimum_location(Esu, H, Vs)
print(H_sediment, Vs_sediment)

### Repeat grid search on crust properties (2nd iteration)

In [ ]:
sediment_props = LayerProps(2.1, Vs_sediment, 1.97, H_sediment)
sediment_props

In [ ]:
H, Vs = np.meshgrid(np.linspace(25, 45, 201), np.linspace(3.0, 5.0, 201))
Esu = np.zeros(H.shape)

In [ ]:
t0 = time.perf_counter()
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Crust loop'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [sediment_props, LayerProps(6.1, _Vs, 2.7, _H)],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

total_time += time.perf_counter() - t0

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=50, cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=10, colors='k', linewidths=1, antialiased=True)
plt.xlabel('Crust $V_s$ (km/s)', fontsize=14)
plt.ylabel('Crust $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('NE68 Crust properties (2nd iteration)', fontsize=20, y=1.05)
plt.savefig('NE68_crust_props_iteration2.png', dpi=300)
plt.show()

In [ ]:
# Extract final minimum H_crust and Vs_crust
H_crust, Vs_crust = find_energy_minimum_location(Esu, H, Vs)
print(H_crust, Vs_crust)

In [ ]:
print("Total time = {}".format(total_time))

### Repeat analysis on Tao's NE68 data using objective function minimization

In [ ]:
import scipy.optimize as optimize

In [ ]:
def objective_fn(model, v0, f_s, p, mantle, Vp, rho, flux_window):
    num_layers = len(model)//2
    earth_model = []
    for i in range(num_layers):
        earth_model.append(LayerProps(Vp[i], model[2*i + 1], rho[i], model[2*i]))
    earth_model = np.array(earth_model)
    energy, _, _ = compute_su_energy(v0, f_s, p, mantle, earth_model, flux_window=flux_window)
    return energy

In [ ]:
# 2-layer model
t0 = time.perf_counter()
Vp = [2.1, 6.1]
rho = [1.97, 2.7]
fixed_args = (v0, f_s, p, mantle_props, Vp, rho, FLUX_WINDOW)
model_0 = np.array([0.5, 0.5, 35.0, 3.7]) # H_0, Vs_0, H_1, Vs_1
bounds = optimize.Bounds([0, 0, 25, 3.0], [1, 1, 45, 5.0])
soln = optimize.minimize(objective_fn, model_0, fixed_args, bounds=bounds)
total_time = time.perf_counter() - t0
print(soln.success, soln.nit)

In [ ]:
H_sediment, Vs_sediment, H_crust, Vs_crust = soln.x

In [ ]:
print(H_sediment, Vs_sediment, H_crust, Vs_crust)

In [ ]:
print("Total time = {}".format(total_time))

----------------------------------------

## Run on real seismic data

In [ ]:
network = 'OA'
target_station = 'CG21'

In [ ]:
# Resampling rate
f_s = 10.0  # Matches dt==0.1 used by Tao
# Time window of original data to use for processing. All traces must have at least this extent
# about the onset time.
TIME_WINDOW = (-20, 50)
# Narrower time window used for integration of energy flux
FLUX_WINDOW = (-10, 20)
# Cut window for selecting central wavelet
CUT_WINDOW = (-5, 30)

In [ ]:
src_file = (r"/g/data/ha3/am7399/shared/OA_RF_analysis/" +
            r"OA_event_waveforms_for_rf_20170911T000036-20181128T230620_rev8.h5")
# src_file = (r"/home/andrew/dev/hiperseis/seismic/receiver_fn/DATA/" +
#             r"OA_event_waveforms_for_rf_20170911T000036-20181128T230620_rev8.h5")

In [ ]:
traces = []
for tr in obspyh5.iterh5(src_file, group='/waveforms/{}.{}.0M'.format(network, target_station), mode='r'):
    traces.append(tr)

In [ ]:
# Group triplets of traces for same event id
data_all = defaultdict(obspy.Stream)
for tr in traces:
    data_all[tr.stats.event_id].append(tr.copy())
data_all = OrderedDict(sorted(data_all.items(), key=lambda k: k[0]))

In [ ]:
# Trim streams and re-order traces into ZNE order.
for evid, stream in data_all.items():
    stream.trim(stream[0].stats.onset + TIME_WINDOW[0],
                stream[0].stats.onset + TIME_WINDOW[1])
    stream.sort(keys=['channel'], reverse=True)

In [ ]:
len(data_all)

In [ ]:
# Apply curation to streams prior to rotation
logger = logging.getLogger(__name__)
discard_ids = []
for evid, stream in data_all.items():
    if not curate_stream3c(evid, stream, logger):
        discard_ids.append(evid)

for evid in discard_ids:
    data_all.pop(evid)

In [ ]:
len(data_all)

In [ ]:
# Rotate to ZRT coordinates
for evid, stream in data_all.items():
    stream.rotate('NE->RT')

In [ ]:
# Detrend the traces
for evid, stream in data_all.items():
    stream.detrend('linear')

In [ ]:
# Run high pass filter to remove high amplitude, low freq noise, if present.
f_min = 0.05
for stream in data_all.values():
    stream.filter('highpass', freq=f_min, corners=2, zerophase=True)

In [ ]:
# Compute SNR of Z component to use as a quality metric
for evid, stream in data_all.items():
    # Taper the traces before SNR computation
    stream.taper(0.05)
    compute_vertical_snr(stream)

In [ ]:
snrs = np.array([s[0].stats.snr_prior for _, s in data_all.items()])

plt.hist(snrs, bins=np.linspace(0, 10, 21))
plt.show()

In [ ]:
# Filter by SNR
discard_ids = []
for evid, stream in data_all.items():
    if stream[0].stats.snr_prior < 3.0:
        discard_ids.append(evid)
        
for evid in discard_ids:
    data_all.pop(evid)

# It does not make sense to filter by similarity, since these are raw waveforms,
# not RFs, and the waveform will be dominated by the source waveform.

In [ ]:
len(data_all)

In [ ]:
# Filter streams with incorrect number of traces
num_pts = np.array([tr.stats.npts for st in data_all.values() for tr in st])
expected_pts = stats.mode(num_pts)[0][0]
expected_pts

In [ ]:
discard_ids = []
for evid, stream in data_all.items():
    if ((stream[0].stats.npts != expected_pts) or
        (stream[1].stats.npts != expected_pts) or
        (stream[2].stats.npts != expected_pts)):
        discard_ids.append(evid)
        
for evid in discard_ids:
    data_all.pop(evid)

In [ ]:
len(data_all)

In [ ]:
# Filter streams with spuriously high amplitude
MAX_AMP = 10000
discard_ids = []
for evid, stream in data_all.items():
    if ((np.max(np.abs(stream[0].data)) > MAX_AMP) or
        (np.max(np.abs(stream[1].data)) > MAX_AMP) or
        (np.max(np.abs(stream[2].data)) > MAX_AMP)):
        discard_ids.append(evid)
        
for evid in discard_ids:
    data_all.pop(evid)

In [ ]:
len(data_all)

In [ ]:
# DEBUG visualize processed waveforms. Watch out for low freq/high amplitude noise.
# for d in data_all.values():
#     d.plot()
#     plt.show()
#     d[2].spectrogram(wlen=3.2)
#     plt.show()

## Start processing

In [ ]:
# TODO: Update this to use dask instead of numpy, so that results will be computed lazily
# using metaprogramming and graph pruning pre-optimization techniques.

### Extract seismic waveforms

Extract time series for Vr and Vz from data_all and shape into 3D array. First dimension is the event so that the 2nd and 3rd dimensions are the wave component (r and z) and time axis respectively. This choice of data layout is made for compatibility with numpy broadcast rules, which applies matrix operations to the last two dimensions and treats the first dimension as an ensemble stack.

In [ ]:
# Note here that we negate the z-component, since this method treats as +z as downwards (increasing depth).
# V0 represents P-SV signal at the surface, i.e. that recorded by surface seismometer.
v0 = _streamdict_to_array(data_all, f_s, TIME_WINDOW, CUT_WINDOW)
v0.shape

In [ ]:
# Get ray params for the events
p = np.array([st[0].stats.slowness/KM_PER_DEG for st in data_all.values()])
p.shape

In [ ]:
# Define bulk properties of mantle (lowermost half-space)
mantle_props = LayerProps(vp=8.0, vs=4.5, rho=3.3, thickness=np.Infinity)

---------------------------------------------------------------

## Plot of grid search over H,Vs space

In [ ]:
# Assumed global property constants
Vp_c = 6.1
Vp_s = 2.1
rho_c = 2.7
rho_s = 1.97

### Perform grid search on sediment properties

In [ ]:
crust_props = LayerProps(Vp_c, 3.7, rho_c, 35)

In [ ]:
H, Vs = np.meshgrid(np.linspace(0, 1.0, 51), np.linspace(0.3, 1.3, 51))
Esu = np.zeros(H.shape)

In [ ]:
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Sediment loop 1'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [LayerProps(Vp_s, _Vs, rho_s, _H), crust_props],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=50, cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=10, colors='k', linewidths=1, antialiased=True)
plt.xlabel('Sediment $V_s$ (km/s)', fontsize=14)
plt.ylabel('Sediment $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('{}.{} Sediment properties (1st iteration)'.format(network, target_station), fontsize=20, y=1.05)
plt.savefig('{}.{}_sediment_props_iteration1.png'.format(network, target_station), dpi=300)
plt.show()

In [ ]:
H_sediment, Vs_sediment = find_energy_minimum_location(Esu, H, Vs)
print(H_sediment, Vs_sediment)

### Perform grid search on crust properties

In [ ]:
sediment_props = LayerProps(Vp_s, Vs_sediment, rho_s, H_sediment)
sediment_props

In [ ]:
H, Vs = np.meshgrid(np.linspace(25, 45, 51), np.linspace(3.0, 4.0, 51))
Esu = np.zeros(H.shape)

In [ ]:
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Crust loop'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [sediment_props, LayerProps(Vp_c, _Vs, rho_c, _H)],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=50, cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=10, colors='k', linewidths=1, antialiased=True)
plt.xlabel('Crust $V_s$ (km/s)', fontsize=14)
plt.ylabel('Crust $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('{}.{} Crust properties (1st iteration)'.format(network, target_station), fontsize=20, y=1.05)
plt.savefig('{}.{}_crust_props_iteration1.png'.format(network, target_station), dpi=300)
plt.show()

In [ ]:
H_crust, Vs_crust = find_energy_minimum_location(Esu, H, Vs)
print(H_crust, Vs_crust)

In [ ]:
# if target_station == 'CG23':
#     H_crust = 42.0
#     Vs_crust = 3.15

### Repeat grid search on sediment properties (2nd iteration)

In [ ]:
crust_props = LayerProps(Vp_c, Vs_crust, rho_c, H_crust)

In [ ]:
H, Vs = np.meshgrid(np.linspace(0, 1.0, 51), np.linspace(0.3, 1.3, 51))
Esu = np.zeros(H.shape)

In [ ]:
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Sediment loop'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [LayerProps(Vp_s, _Vs, rho_s, _H), crust_props],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=50, cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=10, colors='k', linewidths=1, antialiased=True)
plt.xlabel('Sediment $V_s$ (km/s)', fontsize=14)
plt.ylabel('Sediment $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('{}.{} Sediment properties (2nd iteration)'.format(network, target_station), fontsize=20, y=1.05)
plt.savefig('{}.{}_sediment_props_iteration2.png'.format(network, target_station), dpi=300)
plt.show()

In [ ]:
H_sediment, Vs_sediment = find_energy_minimum_location(Esu, H, Vs)
print(H_sediment, Vs_sediment)

### Repeat grid search on crust properties (2nd iteration)

In [ ]:
sediment_props = LayerProps(Vp_s, Vs_sediment, rho_s, H_sediment)
sediment_props

In [ ]:
H, Vs = np.meshgrid(np.linspace(25, 45, 51), np.linspace(3.0, 4.0, 51))
Esu = np.zeros(H.shape)

In [ ]:
results = []
for i, (H_arr, Vs_arr) in tqdm(enumerate(zip(H, Vs)), total=H.shape[0], desc='Crust loop'):
    results.extend(Parallel(n_jobs=-1)(delayed(job_caller)(i, j, v0, f_s, p, mantle_props,
                                                           [sediment_props, LayerProps(Vp_c, _Vs, rho_c, _H)],
                                                           FLUX_WINDOW)
                                       for j, (_H, _Vs) in enumerate(zip(H_arr, Vs_arr))))
# end for
for i, j, energy in results:
    Esu[i, j] = energy

In [ ]:
colmap = 'plasma'
fig = plt.figure(figsize=(16, 12))
plt.contourf(Vs, H, Esu, levels=50, cmap=colmap)
cb = plt.colorbar()
plt.contour(Vs, H, Esu, levels=10, colors='k', linewidths=1, antialiased=True)
plt.xlabel('Crust $V_s$ (km/s)', fontsize=14)
plt.ylabel('Crust $H$ (km)', fontsize=14)
plt.tick_params(right=True, labelright=True, which='both')
plt.tick_params(top=True, labeltop=True, which='both')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.minorticks_on()
plt.xlim(np.min(Vs), np.max(Vs))
plt.ylim(np.min(H), np.max(H))
plt.grid(linestyle=':', color="#80808080")
plt.title('{}.{} Crust properties (2nd iteration)'.format(network, target_station), fontsize=20, y=1.05)
plt.savefig('{}.{}_crust_props_iteration2.png'.format(network, target_station), dpi=300)
plt.show()

In [ ]:
# Extract final minimum H_crust and Vs_crust
H_crust, Vs_crust = find_energy_minimum_location(Esu, H, Vs)
print(H_crust, Vs_crust)

In [ ]:
# Compute energy of solution
solution_model = [LayerProps(Vp_s, Vs_sediment, rho_s, H_sediment),
                  LayerProps(Vp_c, Vs_crust, rho_c, H_crust)]
energy, _, _ = compute_su_energy(v0, f_s, p, mantle_props, solution_model, TIME_WINDOW, FLUX_WINDOW)
print('Original method final energy flux = {}'.format(energy))

### Repeat analysis on OA real data using objective function minimization

In [ ]:
import scipy.optimize as optimize

In [ ]:
def objective_fn(model, v0, f_s, p, mantle, Vp, rho, flux_window):
    num_layers = len(model)//2
    earth_model = []
    for i in range(num_layers):
        earth_model.append(LayerProps(Vp[i], model[2*i + 1], rho[i], model[2*i]))
    earth_model = np.array(earth_model)
    energy, _, _ = compute_su_energy(v0, f_s, p, mantle, earth_model, flux_window=flux_window)
    return energy

In [ ]:
# 2-layer model
Vp = [Vp_s, Vp_c]
rho = [rho_s, rho_c]
k_min, k_max = (1.5, 2.1)
fixed_args = (v0, f_s, p, mantle_props, Vp, rho, FLUX_WINDOW)
model_0 = np.array([0.5, 0.7, 35.0, Vp_c/np.mean((k_min, k_max))]) # H_0, Vs_0, H_1, Vs_1
bounds = optimize.Bounds([0, 0.3, 25, Vp_c/k_max], [1, 1.3, 55, Vp_c/k_min])
soln = optimize.minimize(objective_fn, model_0, fixed_args, bounds=bounds)

In [ ]:
print('Success = {}, Iterations = {}, Function evaluations = {}'.format(soln.success, soln.nit, soln.nfev))

In [ ]:
H_sediment, Vs_sediment, H_crust, Vs_crust = soln.x

In [ ]:
print(H_sediment, Vs_sediment, H_crust, Vs_crust)

In [ ]:
# Compute energy of solution
solution_model = [LayerProps(Vp_s, Vs_sediment, rho_s, H_sediment),
                  LayerProps(Vp_c, Vs_crust, rho_c, H_crust)]
energy, _, _ = compute_su_energy(v0, f_s, p, mantle_props, solution_model, TIME_WINDOW, FLUX_WINDOW)
print('Optimized method final energy flux = {}'.format(energy))

---------------------